In [1]:
# https://resources.oreilly.com/examples/9780596529321/tree/master

class crawler:
    # Initialize the crawler with the name of database.
    def __init__(self, db):
        pass
    
    def __del__(self):
        pass

    def commit(self):
        pass
    
    # Auxialliary function for getting an entry id and adding it if it is not present.
    def get_entry_id(self, table, field, value, create_new = True):
        return None
    
    # Index an individual page.
    def add_to_index(self, url, soup):
        print(f'indexing {url}')
    
    # Extract the text from the HTML page (no tags).
    def get_text_only(self, soup):
        return None
    
    # Separate the words by non-whitespace character.
    def separate_words(self, text):
        return None
    
    # Return true if this url is already indexed.
    def is_indexed(self, url):
        return False
    
    # Add a link between pages.
    def add_link_ref(self, url_from, url_to, link_text):
        pass

    # Starting with a list of pages, do a breadth first search to the given depth,
    # indexing pages as we go.
    def crawl(self, pages, depth = 2):
        pass
    
    # Create the database tables.
    def create_index_tables(self):
        pass

In [2]:
import urllib3
http = urllib3.PoolManager()
url = 'https://en.wikipedia.org/wiki/Programming_language'
# 'http://kiwitobes.com/wiki/Programming_language.html'
r = http.request('GET', url)
r.status, r.data[:500]

/Users/alextanhongpin/Library/Python/3.7/lib/python/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


(200,
 b'<!DOCTYPE html>\n<html class="client-nojs" lang="en" dir="ltr">\n<head>\n<meta charset="UTF-8"/>\n<title>Programming language - Wikipedia</title>\n<script>document.documentElement.className=document.documentElement.className.replace(/(^|\\s)client-nojs(\\s|$)/,"$1client-js$2");RLCONF={"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"Programming_language","wgTitle":"Programming language","wgCurRevisionId":900725449,"wgRevisionId":900725449,"wgArticleId":23015')

In [3]:
import nltk
stopwords = nltk.corpus.stopwords.words('english')
stopwords[:10]

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]

In [4]:
import re
import urllib3
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import sqlite3

http = urllib3.PoolManager()
url = 'https://en.wikipedia.org/wiki/Programming_language'
# pip3 install beautifulsoup4

class Crawler:
    # Initialize the crawler with the name of database.
    def __init__(self, db):
        self.conn = sqlite3.connect(db)
    
    def __del__(self):
        self.conn.close()

    def commit(self):
        self.conn.commit()
    
    # Auxialliary function for getting an entry id and adding it if it is not present.
    def get_entry_id(self, table, field, value, create_new = True):
        c = self.conn.cursor()
        res = c.execute(f'select rowid from {table} where {field} = ?', (value,)).fetchone()
        if res == None:
            stmt = f'insert into {table} ({field}) values (?)'
            r = self.conn.execute(stmt, (value,))
            return r.lastrowid
        else:
            return res[0]
    
    # Index an individual page.
    def add_to_index(self, url, soup):
        if self.is_indexed(url): return
        print(f'indexing {url}')
        
        # Get the individual words.
        text = self.get_text_only(soup)
        words = self.separate_words(text)
        
        # Get the URL id.
        url_id = self.get_entry_id('urllist', 'url', url)
        
        # Link each word to this url.
        for i in range(len(words)):
            word = words[i]
            if word in stopwords: continue
            word_id = self.get_entry_id('wordlist', 'word', word)
            self.conn.execute('insert into wordlocation(url_id, word_id, location) values (?, ?, ?)', (url_id, word_id, i))
    
    # Extract the text from the HTML page (no tags).
    def get_text_only(self, soup):
        return soup.get_text()
#         v = soup.string
#         if v == None:
#             c = soup.contents
#             result_text = ''
#             for t in c:
#                 subtext = self.get_text_only(t)
#                 result_text += subtext + '\n'
#             return result_text
#         else:
#             return v.strip()
    
    # Separate the words by non-whitespace character.
    def separate_words(self, text):
        # splitter = re.compile('\\W*')
        splitter=re.compile('\W+')
        return [s.lower() for s in splitter.split(text) if s != '']
    
    # Return true if this url is already indexed.
    def is_indexed(self, url):
        cur = self.conn.cursor()
        res = cur.execute('select url from urllist where url = ?', (url,)).fetchone()
        if res != None:
            # Check if it has actually been crawled.
            cur = self.conn.cursor()
            c = cur.execute('select * from wordlocation where url_id = ?', (res[0],))
            if c != None:
                res = c.fetchone()
                if res != None: return True
        return False
    
    # Add a link between pages.
    def add_link_ref(self, url_from, url_to, link_text):
        words = self.separate_words(link_text)
        from_id = self.get_entry_id('urllist', 'url', url_from)
        to_id = self.get_entry_id('urllist', 'url', url_to)
        if from_id == to_id: return
        c = self.conn.cursor()
        res = c.execute('insert into link(from_id,to_id) values (?, ?)', (from_id, to_id))
        link_id = res.lastrowid
        for word in words:
            if word in stopwords: continue
            word_id = self.get_entry_id('wordlist', 'word', word)
            self.conn.execute('insert into linkwords(link_id, word_id) values (?, ?)', (link_id, word_id))
    # Starting with a list of pages, do a breadth first search to the given depth,
    # indexing pages as we go.
    def crawl(self, pages, depth = 2):
        for i in range(depth):
            new_pages = set()
            for page in pages:
                try:
                    r = http.request('GET', page)
                    # r.status, r.data 
                    print(f'loaded page {page} {r.status}')
                except:
                    print(f'could not open page {page}')
                    continue
                soup = BeautifulSoup(r.data, 'html.parser')
                self.add_to_index(page, soup)
                
                links = soup.find_all('a')
                for link in links:
                    url = urljoin(page, link.get('href'))
                    # if url.find("'") != -1: continue
                    url = url.split('#')[0] # Remove location portion.
                    if url[0:5] == 'https' and not self.is_indexed(url):
                        print(f'indexing {url}')
                        new_pages.add(url)
                    link_text = self.get_text_only(link)
                    self.add_link_ref(page, url, link_text)
                self.commit()
            pages = new_pages
                
    # Create the database tables.
    def create_index_tables(self):
        self.conn.execute('create table if not exists urllist(url)')
        self.conn.execute('create table if not exists wordlist(word)')
        self.conn.execute('create table if not exists wordlocation(url_id, word_id, location)')
        self.conn.execute('create table if not exists link(from_id integer, to_id integer)')
        self.conn.execute('create table if not exists linkwords(word_id, link_id)')
        self.conn.execute('create index if not exists wordidx on wordlist(word)')
        self.conn.execute('create index if not exists urlidx on urllist(url)')
        self.conn.execute('create index if not exists wordurlidx on wordlocation(word_id)')
        self.conn.execute('create index if not exists urltoidx on link(to_id)')
        self.conn.execute('create index if not exists urlfromidx on link(from_id)')
        self.commit()

In [5]:
crawler = Crawler('searchindex.db')
crawler.create_index_tables()
crawler.crawl([
    'https://en.wikipedia.org/wiki/Programming_language',
    'https://en.wikipedia.org/wiki/Categorical_list_of_programming_languages.html',
    'https://en.wikipedia.org/wiki/Functional_programming'
])

/Users/alextanhongpin/Library/Python/3.7/lib/python/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


loaded page https://en.wikipedia.org/wiki/Programming_language 200
indexing https://en.wikipedia.org/wiki/Programming_language
indexing https://en.wikipedia.org/wiki/Programming_language
indexing https://en.wikipedia.org/wiki/Wikipedia:Protection_policy
indexing https://en.wikipedia.org/wiki/Wikipedia:Pending_changes
indexing https://en.wikipedia.org/w/index.php?title=Special:Log&type=review&page=Programming_language
indexing https://en.wikipedia.org/wiki/Programming_language
indexing https://en.wikipedia.org/wiki/Programming_language
indexing https://en.wikipedia.org/wiki/File:C_Hello_World_Program.png
indexing https://en.wikipedia.org/wiki/File:C_Hello_World_Program.png
indexing https://en.wikipedia.org/wiki/Source_code
indexing https://en.wikipedia.org/wiki/C_(programming_language)
indexing https://en.wikipedia.org/wiki/Compiled
indexing https://en.wikipedia.org/wiki/%22Hello,_World!%22_program
indexing https://en.wikipedia.org/wiki/Formal_language
indexing https://en.wikipedia.org/

indexing https://en.wikipedia.org/wiki/Compiler
indexing https://en.wikipedia.org/wiki/C%2B%2B
indexing https://en.wikipedia.org/wiki/Fortran
indexing https://en.wikipedia.org/wiki/Reference_implementation
indexing https://en.wikipedia.org/wiki/Meta-circular_evaluator
indexing https://en.wikipedia.org/wiki/Prolog
indexing https://en.wikipedia.org/wiki/REXX
indexing https://en.wikipedia.org/wiki/Programming_language
indexing https://en.wikipedia.org/w/index.php?title=Programming_language&action=edit&section=18
indexing https://en.wikipedia.org/wiki/Programming_language_implementation
indexing https://en.wikipedia.org/wiki/Compiler
indexing https://en.wikipedia.org/wiki/Interpreter_(computing)
indexing https://en.wikipedia.org/wiki/Compiler
indexing https://en.wikipedia.org/wiki/BASIC
indexing https://en.wikipedia.org/wiki/Programming_language
indexing https://en.wikipedia.org/wiki/Wikipedia:NOTRS
indexing https://en.wikipedia.org/wiki/Just-in-time_compilation
indexing https://en.wikiped

indexing https://en.wikipedia.org/wiki/Programming_language
indexing https://en.wikipedia.org/wiki/Luca_Cardelli
indexing https://en.wikipedia.org/wiki/Peter_Wegner
indexing https://web.archive.org/web/20060619072646/http://citeseer.ist.psu.edu/cardelli85understanding.html
indexing https://en.wikipedia.org/wiki/Programming_language
indexing https://en.wikipedia.org/wiki/International_Standard_Book_Number
indexing https://en.wikipedia.org/wiki/Special:BookSources/1-86189-080-X
indexing https://en.wikipedia.org/wiki/Programming_language
indexing https://web.archive.org/web/20060107162045/http://www.levenez.com/lang/
indexing https://en.wikipedia.org/wiki/Programming_language
indexing https://web.archive.org/web/20090903084542/http://www.cs.cornell.edu/info/Projects/Nuprl/cs611/fall94notes/cn2/subsection3_1_3.html
indexing https://en.wikipedia.org/wiki/Programming_language
indexing https://web.archive.org/web/20120426010947/http://www.encyclopediaofmath.org/index.php?title=PL%2FI&oldid=19

indexing https://en.wikipedia.org/wiki/Lisp_(programming_language)
indexing https://en.wikipedia.org/wiki/Lua_(programming_language)
indexing https://en.wikipedia.org/wiki/MATLAB
indexing https://en.wikipedia.org/wiki/Objective-C
indexing https://en.wikipedia.org/wiki/OCaml
indexing https://en.wikipedia.org/wiki/Pascal_(programming_language)
indexing https://en.wikipedia.org/wiki/Perl
indexing https://en.wikipedia.org/wiki/PHP
indexing https://en.wikipedia.org/wiki/Python_(programming_language)
indexing https://en.wikipedia.org/wiki/R_(programming_language)
indexing https://en.wikipedia.org/wiki/Ruby_(programming_language)
indexing https://en.wikipedia.org/wiki/Rust_(programming_language)
indexing https://en.wikipedia.org/wiki/Scala_(programming_language)
indexing https://en.wikipedia.org/wiki/Shell_script
indexing https://en.wikipedia.org/wiki/Smalltalk
indexing https://en.wikipedia.org/wiki/Swift_(programming_language)
indexing https://en.wikipedia.org/wiki/TypeScript
indexing https:

...

/Users/alextanhongpin/Library/Python/3.7/lib/python/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


loaded page https://web.archive.org/web/20100715042920/http://www.math.grin.edu/~rebelsky/Courses/CS302/99S/Outlines/outline.02.html 200
indexing https://web.archive.org/web/20100715042920/http://www.math.grin.edu/~rebelsky/Courses/CS302/99S/Outlines/outline.02.html
indexing https://web.archive.org/web/
indexing https://web.archive.org/web/20100715042920/http://www.math.grin.edu/~rebelsky/Courses/CS302/99S/Outlines/outline.02.html
indexing https://archive.org/account/login.php
indexing https://web.archive.org/web/20100715042920/http://www.math.grin.edu/~rebelsky/Courses/CS302/99S/Outlines/outline.02.html
indexing https://web.archive.org/web/20100715042920/http://www.math.grin.edu/~rebelsky/Courses/CS302/99S/Outlines/outline.02.html
indexing https://web.archive.org/web/20100715042920/http://www.math.grin.edu/~rebelsky/Courses/CS302/99S/Outlines/outline.02.html
indexing https://web.archive.org/web/20090703181408/http://www.math.grin.edu:80/~rebelsky/Courses/CS302/99S/Outlines/outline.02.

/Users/alextanhongpin/Library/Python/3.7/lib/python/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


loaded page https://en.wikipedia.org/wiki/Compcert 200
indexing https://en.wikipedia.org/wiki/Compcert
indexing https://en.wikipedia.org/wiki/Compcert
indexing https://en.wikipedia.org/w/index.php?title=Compcert&redirect=no
indexing https://en.wikipedia.org/wiki/Compcert
indexing https://en.wikipedia.org/wiki/Compcert
indexing https://en.wikipedia.org/w/index.php?title=CompCert&action=edit
indexing https://en.wikipedia.org/wiki/Talk:CompCert
indexing https://en.wikipedia.org/wiki/Help:Maintenance_template_removal
indexing https://en.wikipedia.org/wiki/File:Question_book-new.svg
indexing https://en.wikipedia.org/wiki/Wikipedia:Verifiability
indexing https://en.wikipedia.org/wiki/Wikipedia:No_original_research
indexing https://en.wikipedia.org/wiki/Wikipedia:No_original_research
indexing https://en.wikipedia.org/wiki/Help:Maintenance_template_removal
indexing https://en.wikipedia.org/wiki/File:Wiki_letter_w_cropped.svg
indexing https://en.wikipedia.org/w/index.php?title=CompCert&action=e

KeyboardInterrupt: 

In [6]:
[row for row in crawler.conn.execute('select rowid from wordlocation where word_id = 1')]

[(1,),
 (25,),
 (118,),
 (121,),
 (461,),
 (489,),
 (497,),
 (508,),
 (512,),
 (515,),
 (528,),
 (580,),
 (587,),
 (606,),
 (622,),
 (745,),
 (757,),
 (770,),
 (776,),
 (811,),
 (823,),
 (835,),
 (840,),
 (847,),
 (857,),
 (866,),
 (880,),
 (926,),
 (941,),
 (946,),
 (988,),
 (1003,),
 (1032,),
 (1037,),
 (1041,),
 (1063,),
 (1078,),
 (1087,),
 (1101,),
 (1175,),
 (1184,),
 (1213,),
 (1217,),
 (1225,),
 (1293,),
 (1317,),
 (1399,),
 (1425,),
 (1456,),
 (1498,),
 (1501,),
 (1542,),
 (1545,),
 (1551,),
 (1574,),
 (1584,),
 (1599,),
 (1612,),
 (1626,),
 (1632,),
 (1646,),
 (1648,),
 (1667,),
 (1676,),
 (1681,),
 (1695,),
 (1702,),
 (1722,),
 (1748,),
 (1774,),
 (1803,),
 (1836,),
 (1838,),
 (1862,),
 (1866,),
 (1880,),
 (1883,),
 (1899,),
 (1930,),
 (1950,),
 (1956,),
 (1973,),
 (2012,),
 (2233,),
 (2240,),
 (2396,),
 (2417,),
 (2478,),
 (2492,),
 (2512,),
 (2624,),
 (2854,),
 (3030,),
 (3115,),
 (3246,),
 (3281,),
 (3315,),
 (3318,),
 (3337,),
 (3349,),
 (3415,),
 (3426,),
 (3434,),
 (34

In [18]:
class Searcher:
    def __init__(self, db):
        self.conn = sqlite3.connect(db)
    
    def __del__(self):
        self.conn.close()
    
    def get_match_rows(self, q):
        # Strings to build the query.
        fieldlist = 'w0.url_id'
        tablelist = ''
        clauselist = ''
        wordids = []
        
        # Split the words by spaces.
        words = q.split(' ')
        tablenumber = 0
        
        for word in words:
            # Get the word id.
            c = self.conn.cursor()
            wordrow = c.execute('select rowid from wordlist where word=?', (word,)).fetchone()
            if wordrow != None:
                wordid = wordrow[0]
                wordids.append(wordid)
                if tablenumber > 0:
                    tablelist += ','
                    clauselist += ' and '
                    clauselist += f'w{tablenumber-1}.url_id=w{tablenumber}.url_id and '
                fieldlist += f',w{tablenumber}.location'
                tablelist += f'wordlocation w{tablenumber}'
                clauselist += f'w{tablenumber}.word_id = {wordid}'
                tablenumber += 1
        # Create the query from the separate parts.
        fullquery = f'select {fieldlist} from {tablelist} where {clauselist}'
        print(fullquery)
        c = self.conn.cursor()
        res = c.execute(fullquery)
        rows = [row for row in res]
        return rows, wordids

In [19]:
engine = Searcher('searchindex.db')
engine.get_match_rows('functional programming')

select w0.url_id,w0.location,w1.location from wordlocation w0,wordlocation w1 where w0.word_id = 706 and w0.url_id=w1.url_id and w1.word_id = 1


([(1, 1943, 0),
  (1, 1943, 26),
  (1, 1943, 134),
  (1, 1943, 137),
  (1, 1943, 477),
  (1, 1943, 521),
  (1, 1943, 535),
  (1, 1943, 553),
  (1, 1943, 559),
  (1, 1943, 564),
  (1, 1943, 585),
  (1, 1943, 672),
  (1, 1943, 685),
  (1, 1943, 725),
  (1, 1943, 752),
  (1, 1943, 906),
  (1, 1943, 928),
  (1, 1943, 949),
  (1, 1943, 957),
  (1, 1943, 1014),
  (1, 1943, 1034),
  (1, 1943, 1055),
  (1, 1943, 1061),
  (1, 1943, 1071),
  (1, 1943, 1089),
  (1, 1943, 1100),
  (1, 1943, 1122),
  (1, 1943, 1198),
  (1, 1943, 1217),
  (1, 1943, 1222),
  (1, 1943, 1279),
  (1, 1943, 1300),
  (1, 1943, 1350),
  (1, 1943, 1355),
  (1, 1943, 1361),
  (1, 1943, 1394),
  (1, 1943, 1421),
  (1, 1943, 1437),
  (1, 1943, 1456),
  (1, 1943, 1574),
  (1, 1943, 1587),
  (1, 1943, 1632),
  (1, 1943, 1637),
  (1, 1943, 1649),
  (1, 1943, 1756),
  (1, 1943, 1799),
  (1, 1943, 1938),
  (1, 1943, 1983),
  (1, 1943, 2039),
  (1, 1943, 2105),
  (1, 1943, 2109),
  (1, 1943, 2172),
  (1, 1943, 2176),
  (1, 1943, 218

In [17]:
[row for row in engine.conn.cursor().execute('select * from wordlist limit 10')]

[('programming',),
 ('language',),
 ('wikipedia',),
 ('document',),
 ('documentelement',),
 ('classname',),
 ('replace',),
 ('client',),
 ('nojs',),
 ('1client',)]

In [ ]:
# Content-based Ranking